In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-vs-human-text/AI_Human.csv


In [19]:
# Load data from CSV
df = pd.read_csv('/kaggle/input/ai-vs-human-text/AI_Human.csv')

In [21]:
# Extract texts and labels
texts = df['text'].tolist()
labels = df['generated'].values

In [23]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [24]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

In [25]:
from keras.src.legacy.preprocessing.text import Tokenizer

In [26]:
# Tokenizing the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [27]:
from keras.src.utils import pad_sequences

In [28]:
# Padding sequences
max_len = max([len(seq) for seq in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [17]:
# Building the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 64, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [29]:
# Compiling the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train_pad, y_train, epochs=4, batch_size=4)

Epoch 1/4
   24/85266 ━━━━━━━━━━━━━━━━━━━━ 9:47 7ms/step - accuracy: 0.4890 - loss: 0.6931

I0000 00:00:1715013574.469489     115 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


70647/85266 ━━━━━━━━━━━━━━━━━━━━ 1:36 7ms/step - accuracy: 0.9803 - loss: 0.0507

In [ ]:
# Save the model
model.save("cnn_model.h5")